### 雪花算法原理
雪花算法（Snowflake）是一种分布式系统中生成全局唯一 ID的轻量级算法，核心是通过固定结构的 64 位二进制数，在不依赖中心化存储的前提下，保证 ID 的唯一性、有序性和高效生成，常用于分布式数据库、微服务等场景。

| 段名称            | 位数  | 作用说明                                                     |
| ----------------- | ----- | ------------------------------------------------------------ |
| 符号位            | 1 位  | 固定为 0，确保 ID 是正整数（二进制最高位为 1 表示负数，此处无用）。 |
| 时间戳段          | 41 位 | 记录**相对于 “起始时间戳” 的毫秒数**（非当前绝对时间），可支撑约 69 年（2^41-1 ≈ 69 年）。 |
| 机器 / 节点 ID 段 | 10 位 | 区分不同的服务器节点（或进程），最多支持 2^10=1024 个节点（可按需调整位数）。 |
| 序列号段          | 12 位 | 同一节点、同一毫秒内生成的 ID 计数器，最多支持 2^12=4096 个 ID / 毫秒（避免同毫秒内重复）。 |

### 时间回拨

雪花算法生成 ID 时，会依赖服务器的系统物理时间（作为 ID 的 “时间戳” 部分，保证 ID 整体按时间递增）。
“时间回拨” 就是服务器的系统时间突然倒退（比如：原本系统时间是 10:00:05，突然变成 10:00:03），导致当前获取的时间戳，比 “上一次生成 ID 时记录的时间戳” 还小。
这会直接引发问题：若用倒退后的时间戳生成 ID，可能和 “上一次 10:00:03 生成的 ID” 重复，破坏 ID 唯一性。

在雪花算法中，额外记录 “上一次生成 ID 的时间戳”（记为last_time）。
每次生成新 ID 时，对比 “当前系统时间戳”（current_time）和last_time：
若current_time ≥ last_time：正常生成 ID，更新last_time为current_time；
若current_time < last_time：判定为 “时间回拨”，触发容错逻辑。

根据 “回拨的时间差”（last_time - current_time），分 3 种场景处理：
- 短回拨（比如差≤1 秒）：
不换时间戳，继续用last_time（避免时间倒退），只递增 “ID 的序列号部分”（雪花算法默认 12 位序列号，1 毫秒能生成 4096 个唯一序列号），直到current_time追平last_time。
例：上次last_time=1000ms，当前回拨到 999ms，就用 1000ms 当时间戳，序列号从之前的 100 变成 101，保证 ID 唯一。
- 中等回拨（比如差 1~5 秒）：
暂时 “阻塞等待”，直到current_time追平last_time（比如等 5 秒，直到系统时间回到 1000ms），再按正常逻辑生成 ID（更新last_time，重置序列号）。
代价是短暂阻塞，适合对性能要求不极端的场景。
- 长回拨（比如差 > 5 秒）：
直接抛出 “时间回拨异常”，拒绝生成 ID（避免长时间阻塞影响业务），同时触发报警（通知运维排查时间问题）；也可备用方案：用 “内部逻辑时间”（不依赖物理时间，只单调递增）临时生成 ID，保证业务不中断。


